In [ ]:
import torch
from torch import nn

In [ ]:
cfgs = { "A": [64, "M", 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"],
         "B": [64, 64, "M", 128, 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"],
         "D": [64, 64, "M", 128, 128, "M", 256, 256, 256, "M", 512, 512, 512, "M", 512, 512, 512, "M"],
         "E": [64, 64, "M", 128, 128, "M", 256, 256, 256, 256, "M", 512, 512, 512, 512, "M", 512, 512, 512, 512, "M"] }

In [ ]:
class VGG(nn.Module):
    def __init__(self, cfg, batch_norm = False, num_classes = 1000, init_weights = True, drop_p = 0.5):
        super().__init__()

        self.features = self.make_layers(cfg, batch_norm)
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7)) # 7x7 이 되도록 avg pooling 하는 녀석
        self.classifier = nn.Sequential(nn.Linear(512 * 7 * 7, 4096),
                                        nn.ReLU(),
                                        nn.Dropout(p=drop_p),
                                        nn.Linear(4096, 4096),
                                        nn.ReLU(),
                                        nn.Dropout(p=drop_p),
                                        nn.Linear(4096, num_classes))

        if init_weights:
            for m in self.modules():
                if isinstance(m, nn.Conv2d):
                    nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
                    # mode="fan_out" → backward gradient 분산 유지를 위해 fan_out 기준으로 std 계산
                    # nonlinearity="relu" → ReLU가 반 깎으니까 분산도 반으로 줄어서 gain=√2로 분산을 보정
                    # 즉, 사용한 분산 값은 2/fan_out
                    if m.bias is not None:
                        nn.init.constant_(m.bias, 0)
                elif isinstance(m, nn.Linear):
                    nn.init.normal_(m.weight, mean=0, std=0.01)
                    nn.init.constant_(m.bias, 0)

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

    def make_layers(self, cfg, batch_norm = False):
        layers = []
        in_channels = 3
        for v in cfg: # cfg = [64, 64, "M", 128, 128, "M", 256, 256, 256, "M", 512, 512, 512, "M", 512, 512, 512, "M"]
            if type(v) == int:
                if batch_norm:
                    layers += [nn.Conv2d(in_channels, v, 3, padding=1, bias=False), # 어차피 BN에 bias 포함
                               nn.BatchNorm2d(v),
                               nn.ReLU()]
                else:
                    layers += [nn.Conv2d(in_channels, v, 3, padding=1),
                               nn.ReLU()]
                in_channels = v
            else:
                layers += [nn.MaxPool2d(2)]

        return nn.Sequential(*layers)

In [ ]:
avgpool = nn.AdaptiveAvgPool2d((4, 4))
print(avgpool(torch.randn(2,3,32,32)).shape)
x = torch.randn(2,3,2,2)
print(x)
print(avgpool(x)) # 작은 놈이 들어오면 늘려서라도 맞춰준다 # 값을 복제 시켜놓음

torch.Size([2, 3, 4, 4])
tensor([[[[ 0.5192, -1.3704],
          [-0.5193, -0.0531]],

         [[-0.0574, -0.8994],
          [ 0.4007, -0.0164]],

         [[-0.6730, -0.4228],
          [ 0.4561, -0.4095]]],


        [[[-0.5832, -0.7258],
          [ 0.9588, -1.1505]],

         [[-0.7809, -0.4652],
          [ 0.5487,  1.2373]],

         [[ 0.8288,  0.9699],
          [ 0.2145,  0.0349]]]])
tensor([[[[ 0.5192,  0.5192, -1.3704, -1.3704],
          [ 0.5192,  0.5192, -1.3704, -1.3704],
          [-0.5193, -0.5193, -0.0531, -0.0531],
          [-0.5193, -0.5193, -0.0531, -0.0531]],

         [[-0.0574, -0.0574, -0.8994, -0.8994],
          [-0.0574, -0.0574, -0.8994, -0.8994],
          [ 0.4007,  0.4007, -0.0164, -0.0164],
          [ 0.4007,  0.4007, -0.0164, -0.0164]],

         [[-0.6730, -0.6730, -0.4228, -0.4228],
          [-0.6730, -0.6730, -0.4228, -0.4228],
          [ 0.4561,  0.4561, -0.4095, -0.4095],
          [ 0.4561,  0.4561, -0.4095, -0.4095]]],


        [[[-0.58

In [ ]:
model = nn.Sequential(nn.Conv2d(3,32,3),
                      nn.ReLU(),
                      nn.Sequential(nn.Conv2d(32,64,3),
                                    nn.ReLU(),
                                    nn.Conv2d(64,64,3),
                                    nn.ReLU()),
                      nn.Conv2d(64,128,3))
[m for m in model.modules()]

[Sequential(
   (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
   (1): ReLU()
   (2): Sequential(
     (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
     (1): ReLU()
     (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
     (3): ReLU()
   )
   (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
 ),
 Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1)),
 ReLU(),
 Sequential(
   (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
   (1): ReLU()
   (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
   (3): ReLU()
 ),
 Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1)),
 ReLU(),
 Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1)),
 ReLU(),
 Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))]

In [ ]:
# model1 = nn.Sequential([nn.Linear(1,1),
#                        nn.Linear(1,1)]) # 리스트를 넣으면 안돼요!

model2 = nn.Sequential(nn.Linear(1,1),
                       nn.Linear(1,1))

# print(*[1,2])
# print([1,2])

# model3 = nn.Sequential(*[nn.Linear(1,1),
#                          nn.Linear(1,1)])

In [ ]:
model = VGG(cfgs["E"], batch_norm=True)
# print(model)
!pip install torchinfo
from torchinfo import summary
summary(model, input_size=(2,3,224,224), device='cpu')

Layer (type:depth-idx)                   Output Shape              Param #
VGG                                      [2, 1000]                 --
├─Sequential: 1-1                        [2, 512, 7, 7]            --
│    └─Conv2d: 2-1                       [2, 64, 224, 224]         1,728
│    └─BatchNorm2d: 2-2                  [2, 64, 224, 224]         128
│    └─ReLU: 2-3                         [2, 64, 224, 224]         --
│    └─Conv2d: 2-4                       [2, 64, 224, 224]         36,864
│    └─BatchNorm2d: 2-5                  [2, 64, 224, 224]         128
│    └─ReLU: 2-6                         [2, 64, 224, 224]         --
│    └─MaxPool2d: 2-7                    [2, 64, 112, 112]         --
│    └─Conv2d: 2-8                       [2, 128, 112, 112]        73,728
│    └─BatchNorm2d: 2-9                  [2, 128, 112, 112]        256
│    └─ReLU: 2-10                        [2, 128, 112, 112]        --
│    └─Conv2d: 2-11                      [2, 128, 112, 112]        147,

In [ ]:
x = torch.randn(2,3,224,224)
print(model(x).shape)

torch.Size([2, 1000])


In [ ]:
x = torch.randn(2,3,300,300)
print(model(x).shape)

torch.Size([2, 1000])


In [ ]:
x = torch.randn(2,3,32,32)
print(model(x).shape)

torch.Size([2, 1000])


In [ ]:
# nn.MaxPool2d(2)(torch.randn(2,3,1,1)) # error!